<a href="https://colab.research.google.com/github/sinahuss/solar-flare-prediction/blob/main/notebooks/solar_flare_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C964 Capstone: Solar Flare Prediction and Analysis

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset from a raw GitHub URL

In [ ]:
url = 'https://raw.githubusercontent.com/sinahuss/solar-flare-prediction/refs/heads/main/data/data.csv'
df = pd.read_csv(url)

# Display the first few rows to confirm it loaded correctly.

In [ ]:
df.head()

# Define and apply a function to create a unified flare class

In [ ]:
# Determine the highest flare class for each row
def get_flare_class(row):
    if row['severe flares'] > 0:
        return 'X'
    elif row['moderate flares'] > 0:
        return 'M'
    elif row['common flares'] > 0:
        return 'C'
    else:
        return 'None'

# Create a new target column
df['flare_class'] = df.apply(get_flare_class, axis=1)

print(df['flare_class'].value_counts())